In [1]:
#Sistemas de recomendação

1) Carregar os dados
2) Pré-processamento dos dados
3) Divisão dos dados entre treinamento e teste
4) Criar o sistema de recomendação através do ALS
5) Realizar previsões
6) Testar o modelo

In [1]:
#importa a biblioteca que cria a seção do spark
from pyspark.sql import SparkSession 

In [2]:
#inicia a seção para a utilização do spark
spark = SparkSession\
    .builder\
    .appName("recomendacao")\
    .getOrCreate() #cria a seção caso não exista ou obtém a já criada

23/01/11 23:22:29 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/11 23:22:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/11 23:22:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#define o diretório que contém o arquivo a ser utilizado
diretorioRecomendacao="./data/u.data"  

1) Carregando o arquivo.

In [4]:
#lendo arquivos armazenados através da função genérica
#formato .data não pode salvar direto em dataframe, por isso usa-se rdd
rdd_movies = spark.sparkContext.textFile(diretorioRecomendacao)

In [5]:
#chama 10 primeiros itens
# user id | item id | rating | timestamp
rdd_movies.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [10]:
#definindo as bibliotecas a serem utilizadas
#MLlib utilizada para implementar os algoritmos ALS e Rating
from pyspark.mllib.recommendation import ALS, Rating  

2) Pré-processamento dos dados
3) Separando os dados para treinamento e teste

In [12]:
#dividindo os dados entre treinamento e teste
(trainRatings, testRatings) = rdd_movies.randomSplit([0.7, 0.3])

In [13]:
trainRatings.take(5)

Out[9]: ['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '166\t346\t1\t886397596',
 '115\t265\t2\t881171488']

In [14]:
testRatings.first()  #print da primeira linha do RDD

Out[10]: '244\t51\t2\t880606923'

In [15]:
trainingData = trainRatings.map(lambda l: l.split('\t')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))  #aplicando a função Rating

In [16]:
trainingData.first()  #print do RDD criado através da função Rating

Out[12]: Rating(user=196, product=242, rating=3.0)

In [17]:
#mesmo procedimento para os dados de teste
testData = testRatings.map(lambda l: l.split('\t')).map(lambda l: (int(l[0]), int(l[1])))

In [18]:
testData.first()  #id do usuário | id do filme

Out[14]: (244, 51)

#4) Construindo o modelo ALS

In [20]:
#definindo as variáveis do modelo
rank = 10  # número de fatores latentes do modelo  R->P (usuários)*Q (itens) => R_mxn = P_mxrank * Q_rankxm  (em que m= numero de usuário e n= numero de itens)
numIterations = 50 #número de iterações realizadas pelo modelo
model = ALS.train(trainingData, rank, numIterations) # treina o modelo

#5) Previsão do modelo

In [22]:
model.predict(253, 465)  # entrada (usuário,produto)

Out[16]: 3.9086929950865077

In [23]:
previsao = model.predictAll(testData)  #previsão para todos os dados de teste
previsao.first()

Out[17]: Rating(user=894, product=1080, rating=2.187004642601909)

In [24]:
previsao = previsao.map(lambda l: ((l[0], l[1]), l[2])) #mapea para a exibição
previsao.take(5)

Out[18]: [((894, 1080), 2.187004642601909),
 ((264, 320), 5.565980638755637),
 ((385, 320), 6.956242876993615),
 ((49, 320), 5.81031221038799),
 ((118, 320), 3.900088577898167)]

In [25]:
testRating2 = testRatings.map(lambda l: l.split('\t')).map(lambda l: ((int(l[0]), int(l[1])), float(l[2]))) # mapea para exibição e utilização na análise

In [26]:
testRating2.first()

Out[21]: ((244, 51), 2.0)

In [27]:
ratingsAndPredictions = testRating2.join(previsao)
ratingsAndPredictions.take(5)

Out[22]: [((234, 1184), (2.0, 3.136279929500878)),
 ((20, 288), (1.0, 3.1563148849880296)),
 ((160, 174), (5.0, 3.608742030692189)),
 ((293, 471), (3.0, 2.690341659498764)),
 ((174, 368), (1.0, 1.2317709660007763))]

#6) Avaliação do modelo

In [29]:
MSE = ratingsAndPredictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print ("Erro médio quadrado = " + str(MSE))

Erro médio quadrado = 1.3917179976332206

In [30]:
#dado um usuário -> recomenda os top 5 produtos
model.recommendProducts(105, 5) 

Out[24]: [Rating(user=105, product=1086, rating=10.404151472284607),
 Rating(user=105, product=1282, rating=10.059519202435673),
 Rating(user=105, product=1129, rating=8.15539879719414),
 Rating(user=105, product=916, rating=8.139311224018458),
 Rating(user=105, product=1192, rating=8.115013085806174)]

In [31]:
#dado um produto -> recomenda top 5 usuários
model.recommendUsers(1, 5)  #filme Toy Story (1995)

Out[25]: [Rating(user=791, product=1, rating=6.663269162167041),
 Rating(user=341, product=1, rating=6.59332322597731),
 Rating(user=359, product=1, rating=5.920746848976035),
 Rating(user=810, product=1, rating=5.86259872350825),
 Rating(user=337, product=1, rating=5.8396218409615575)]

In [32]:
#mostrando o vetor de características referentes usuários (coluna - P)
model.userFeatures().take(1)[0]

Out[26]: (8,
 array('d', [-1.3223897218704224, -0.401091992855072, -0.020178183913230896, -0.654310941696167, 0.0015730439918115735, -0.7683062553405762, -0.1260661780834198, 1.0363516807556152, -1.0391936302185059, -0.14950275421142578]))

In [33]:
#mostra o vetor de características referente a um produto (linha - Q)
model.productFeatures().take(1)[0]

Out[27]: (8,
 array('d', [-1.7562367916107178, 0.6425478458404541, -0.2787933945655823, -0.23034699261188507, 0.9097221493721008, -0.7952987551689148, -0.5979781746864319, 0.8866454362869263, -0.6941563487052917, 0.6015448570251465]))